# Finetune VAE models

## importing the libraries

In [1]:
import os
ml_code_path='/home/leilapirhaji/mz_embed_engine/ml'
os.chdir(ml_code_path)

import pandas as pd
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

import torch
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from get_pretrain_encoder import get_pretrain_input_data

%who

[neptune] [warning] NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.


DataLoader	 LabelEncoder	 TensorDataset	 get_pretrain_input_data	 label_encoder	 ml_code_path	 optim	 os	 pd	 
torch	 


## getting Finetune datasets

In [2]:

input_data_location='/home/leilapirhaji/PROCESSED_DATA_2'
pretrain_save_dir='/home/leilapirhaji/pretrained_VAE_models' 


(X_data_all, y_data_all, X_data_train, y_data_train, X_data_val, y_data_val, X_data_test, y_data_test)=get_pretrain_input_data(input_data_location)


X_data_train.shape, y_data_train.shape, X_data_val.shape, y_data_val.shape, X_data_test.shape, y_data_test.shape


/home/leilapirhaji/mz_embed_engine/ml/get_pretrain_encoder.py:60: DtypeWarning: Columns (1,8,28,30,31,32,33,34,46,50,51,52,53,54,55,56,57,58) have mixed types. Specify dtype option on import or set low_memory=False.
  y_data_all = pd.read_csv(pretrain_y_all, index_col=0)


((14391, 2736),
 (14391, 58),
 (3074, 2736),
 (3074, 58),
 (3083, 2736),
 (3083, 58))

## pre-train VAE models

In [4]:
X_data_train.shape, y_data_train.shape, X_data_val.shape, y_data_val.shape, X_data_test.shape, y_data_test.shape

((14391, 2736),
 (14391, 58),
 (3074, 2736),
 (3074, 58),
 (3083, 2736),
 (3083, 58))

In [5]:
import importlib
import models_VAE
importlib.reload(models_VAE)
from models_VAE import VAE  # Import the VAE model class

import train_pretrain_vae
importlib.reload(train_pretrain_vae)
from train_pretrain_vae import pretrain_vae


# Simulated data for example purposes

pretrain_vae(X_data_train, y_data_train, X_data_val, y_data_val, X_data_test, y_data_test, 
                latent_size=128, 
                num_hidden_layers=3, dropout_rate=0.2, activation='leakyrelu', 
                use_batch_norm=True, epochs=3, learning_rate=1e-3, 
                save_path=pretrain_save_dir, model_name='vae_model.pth')


Epoch 1/3, Loss: 64.60926751030816
Validation Loss: 64.5248378831513
Epoch 2/3, Loss: 64.5556440226237
Validation Loss: 64.50877707345145
Epoch 3/3, Loss: 64.54763275146485
Validation Loss: 64.50805461649992
Model kwargs saved to /home/leilapirhaji/pretrained_VAE_models/model_hyperparameters.json
Model saved to /home/leilapirhaji/pretrained_VAE_models/vae_model.pth


In [27]:
import models_VAE
importlib.reload(models_VAE)
from models_VAE import VAE  # Import the VAE model class

vae = VAE(input_size=2736, latent_size=128, num_hidden_layers=3)
print("Encoder hidden sizes:", vae.encoder.hidden_sizes)
print("Decoder hidden sizes should mirror encoder:", vae.decoder.network)


Encoder hidden sizes: [1242, 563, 256]
Decoder hidden sizes should mirror encoder: Sequential(
  (0): Linear(in_features=128, out_features=256, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Dropout(p=0.2, inplace=False)
  (3): Linear(in_features=256, out_features=563, bias=True)
  (4): LeakyReLU(negative_slope=0.01)
  (5): Dropout(p=0.2, inplace=False)
  (6): Linear(in_features=563, out_features=1242, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): Dropout(p=0.2, inplace=False)
  (9): Linear(in_features=1242, out_features=2736, bias=True)
)


In [6]:
import torch
import json
import os
import models_VAE
importlib.reload(models_VAE)
from models_VAE import VAE  # Import the VAE model class


model_path = f'{pretrain_save_dir}/vae_model.pth'
hyperparameters_path = f'{pretrain_save_dir}/model_hyperparameters.json'
#Step 1: Load hyperparameters from the JSON file
with open(hyperparameters_path, 'r') as f:
    hyperparameters = json.load(f)

# Step 2: Initialize the VAE model using the loaded hyperparameters
vae = VAE(**hyperparameters)

# Step 3: Load the saved model state (weights) into the VAE model
vae.load_state_dict(torch.load(model_path))
    

/var/tmp/ipykernel_3160887/2290562983.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vae.load_state_dict(torch.load(model_path))


<All keys matched successfully>

In [7]:
vae.encoder.hidden_sizes()

TypeError: 'list' object is not callable

In [8]:
print (vae)

VAE(
  (encoder): Dense_Layers(
    (network): Sequential(
      (0): Linear(in_features=2736, out_features=1242, bias=True)
      (1): BatchNorm1d(1242, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
      (3): Dropout(p=0.2, inplace=False)
      (4): Linear(in_features=1242, out_features=563, bias=True)
      (5): BatchNorm1d(563, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): LeakyReLU(negative_slope=0.01)
      (7): Dropout(p=0.2, inplace=False)
      (8): Linear(in_features=563, out_features=256, bias=True)
      (9): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (10): LeakyReLU(negative_slope=0.01)
      (11): Dropout(p=0.2, inplace=False)
      (12): Linear(in_features=256, out_features=256, bias=True)
    )
  )
  (decoder): Dense_Layers(
    (network): Sequential(
      (0): Linear(in_features=128, out_features=256, bias=True)
      (1): BatchNorm1d(25